<a href="https://colab.research.google.com/github/PaoDOrado/Prueba-tecnica-Colombina/blob/main/Prueba_t%C3%A9cnica_Colombina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U plotly
!pip install -U kaleido 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.2 MB 31.0 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79.9 MB 80 kB/s 


In [ ]:
#Importar librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl 
import seaborn as sns
import math
import plotly.graph_objs as go 
import plotly.express as px
import datetime
from pandas.core import groupby
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
%matplotlib inline

# **1. DESCRIPCIÓN DEL PROBLEMA**
---
Colombina está interesado en saber qué clientes tienen mayor probabilidad de no volver a comprar
ninguno de sus productos. El negocio ha establecido que un cliente que no compra durante 2 meses
es un cliente fugado.
 


# **2. ENTENDIMIENTO DE LOS DATOS**
---
  

En esta sección se obtendrán unas estadísticas descriptivas para entender los datos. 


In [ ]:
datosColombina=pd.read_csv('prueba_DS.txt', sep=',')
datosColombina.head(10)

,Doc_vtas,Fecha,Solic,Material,Cnt_uni,sect_mat,Cnt_cart,ValN_Pos,Moneda
0,7503277849,2021-11-01,18134244.0,28424.0,72.0,42.0,1.000,347.76,COP
1,7503277849,2021-11-01,18134244.0,26780.0,6.0,42.0,6.000,781.40,COP
2,7503277849,2021-11-01,18134244.0,28846.0,72.0,42.0,1.000,309.05,COP
3,7503277849,2021-11-01,18134244.0,6572.0,18.0,42.0,0.375,124.59,COP
4,7503277849,2021-11-01,18134244.0,28850.0,1.0,42.0,1.000,478.87,COP
5,7503277849,2021-11-01,18134244.0,28852.0,72.0,42.0,1.000,425.09,COP
6,7503277849,2021-11-01,18134244.0,28854.0,1.0,42.0,1.000,609.77,COP
7,7503277850,2021-11-01,1421200.0,9388.0,6.0,60.0,6.000,4528.80,COP
8,7503277850,2021-11-01,1421200.0,40302.0,6.0,60.0,6.000,2764.80,COP
9,7503277850,2021-11-01,1421200.0,54932.0,9.0,60.0,9.000,4375.08,COP


**Descripción de la base de datos**

El conjunto de datos cuenta con las variables que se describen a continuación:

| Variable | Definición	| Valores | 
| --- | --- | --- |
|Doc_vtas | Id de pedido|Númerico|
|Fecha |Fecha del pedido|Fecha|
|Solic |Código del cliente|Númerico|
|Material |Id del producto solicitado|Númerico|
|Cnt_uni |Cantidad pedida en unidad venta|Númerico|
|sect_mat| código del sector del producto|Númerico|
|Cnt_cart| Cantidad pedidos en unidad cartón|Númerico|
|ValN_Pos| Valor neto en dinero|Númerico|
|Moneda |Moneda|Caracter|


***Descripciones generales***

En esta parte se conocerá como está conformada la base de datos y se realizará un análisis de valores faltantes.

In [ ]:
datosColombina.info()
datosColombina.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8338873 entries, 0 to 8338872
Data columns (total 9 columns):
 #   Column    Dtype  
---  ------    -----  
 0   Doc_vtas  int64  
 1   Fecha     object 
 2   Solic     float64
 3   Material  float64
 4   Cnt_uni   float64
 5   sect_mat  float64
 6   Cnt_cart  float64
 7   ValN_Pos  float64
 8   Moneda    object 
dtypes: float64(6), int64(1), object(2)
memory usage: 572.6+ MB


(8338873, 9)

La base cuenta con 8338873 observaciones y 9 variables.

In [ ]:
#Cantidad de valores faltantes por variable
cont_valores_faltantes = datosColombina.isnull().sum()
cont_valores_faltantes

Doc_vtas    0
Fecha       1
Solic       1
Material    1
Cnt_uni     1
sect_mat    1
Cnt_cart    1
ValN_Pos    1
Moneda      1
dtype: int64

Se encontró un valor faltante.


 ***Estadísticas descriptivas***

Se obtendrán las estadísticas descriptivas de posición y de dispersión (promedio,mediana, desviación estándar, máximo y mínimo) de las variables cuantitativas Cnt_uni, Cnt_cart y ValN_Pos, para describir como se encuentran los datos.

In [ ]:
datosColombina[['Cnt_uni','Cnt_cart','ValN_Pos']].quantile([0.25, 0.5, 0.75])

,Cnt_uni,Cnt_cart,ValN_Pos
0.25,1.0,0.055556,39.60
0.50,1.0,0.083333,53.34
0.75,3.0,0.125000,84.48


In [ ]:
datosColombina[['Cnt_uni','Cnt_cart','ValN_Pos']].mean()

Cnt_uni      3.465728
Cnt_cart     1.110388
ValN_Pos    89.186420
dtype: float64

Para la tres variables anteriores se puede observar que la mediana es menor que el promedio, por lo que se tiene una distribución asimetrica positiva esto quiere decir que los datos tienden hacia valores menores al promedio (media).

---



In [ ]:
datosColombina[['Cnt_uni','Cnt_cart','ValN_Pos']].std()

Cnt_uni     338.076556
Cnt_cart    337.951969
ValN_Pos    489.520831
dtype: float64

Al obtener la desviación estándar, se puede observar que los datos estan muy dispersos, ya que se tiene una desviación grande.

In [ ]:
#valores mínimos de las variables cuantitativas
datosColombina[['Cnt_uni','Cnt_cart','Cnt_cart']].min()

Cnt_uni     0.0
Cnt_cart    0.0
Cnt_cart    0.0
dtype: float64

In [ ]:
#valores máximos de las variables cuantitativas
datosColombina[['Cnt_uni','Cnt_cart','Cnt_cart']].max()

Cnt_uni     200000.0
Cnt_cart    200000.0
Cnt_cart    200000.0
dtype: float64

Para el analisis de fuga primero se realizara el cambio de la variable Fecha a datetime 

In [ ]:
datosColombina['Fecha'] = pd.to_datetime(datosColombina['Fecha'])

In [ ]:
# Creación de las variables FechaInicio y FechaFinal
datosColombina['FechaInicio']=datosColombina['Fecha']
datosColombina['FechaFinal']=datosColombina['Fecha']

In [ ]:
#obtencion de la fecha minima
datosColombina[['Fecha']].min()

Fecha   2021-11-01
dtype: datetime64[ns]

In [ ]:
#Creación de la variable rango despues de fijar una fecha en el tiempo
datosColombina['Rango']=datosColombina['Fecha'].sub(pd.Timestamp('2021-10-01')).dt.days
datosColombina.head()

,Doc_vtas,Fecha,Solic,Material,Cnt_uni,sect_mat,Cnt_cart,ValN_Pos,Moneda,FechaInicio,FechaFinal,Rango
0,7503277849,2021-11-01,18134244.0,28424.0,72.0,42.0,1.000,347.76,COP,2021-11-01,2021-11-01,31.0
1,7503277849,2021-11-01,18134244.0,26780.0,6.0,42.0,6.000,781.40,COP,2021-11-01,2021-11-01,31.0
2,7503277849,2021-11-01,18134244.0,28846.0,72.0,42.0,1.000,309.05,COP,2021-11-01,2021-11-01,31.0
3,7503277849,2021-11-01,18134244.0,6572.0,18.0,42.0,0.375,124.59,COP,2021-11-01,2021-11-01,31.0
4,7503277849,2021-11-01,18134244.0,28850.0,1.0,42.0,1.000,478.87,COP,2021-11-01,2021-11-01,31.0


In [ ]:
#se agrupan los clientes de acuerdo a su fecha de inicio y fin
grupo1=datosColombina.groupby('Solic').agg({'FechaInicio':lambda x:x.min(),'FechaFinal':lambda x:x.max()}).reset_index()
grupo1.head()

,Solic,FechaInicio,FechaFinal
0,49128.0,2021-11-02,2022-07-11
1,69792.0,2021-11-05,2022-07-05
2,69838.0,2022-07-12,2022-07-12
3,69868.0,2021-11-09,2022-07-05
4,70334.0,2022-02-26,2022-05-31


In [ ]:
#Agrupar el rango en un formato set que tiene las diferencias de las fechas 
grupo2=datosColombina.groupby('Solic')['Rango'].apply(set).reset_index()
grupo2.head()

,Solic,Rango
0,49128.0,"{130.0, 259.0, 132.0, 136.0, 143.0, 276.0, 283..."
1,69792.0,"{256.0, 130.0, 137.0, 272.0, 147.0, 277.0, 35...."
2,69838.0,{284.0}
3,69868.0,"{66.0, 130.0, 39.0, 103.0, 263.0, 74.0, 235.0,..."
4,70334.0,"{242.0, 180.0, 148.0}"


In [ ]:
from numpy.ma.core import right_shift
#union de los dos grupos en una sola tabla
datos=pd.merge(grupo1,grupo2[['Rango']],left_index=True,right_index=True,how='outer')
datos.head()

,Solic,FechaInicio,FechaFinal,Rango
0,49128.0,2021-11-02,2022-07-11,"{130.0, 259.0, 132.0, 136.0, 143.0, 276.0, 283..."
1,69792.0,2021-11-05,2022-07-05,"{256.0, 130.0, 137.0, 272.0, 147.0, 277.0, 35...."
2,69838.0,2022-07-12,2022-07-12,{284.0}
3,69868.0,2021-11-09,2022-07-05,"{66.0, 130.0, 39.0, 103.0, 263.0, 74.0, 235.0,..."
4,70334.0,2022-02-26,2022-05-31,"{242.0, 180.0, 148.0}"


In [ ]:
#bucle que itera dentro de cada elemento de la lista para obtener las diferencias,media,desviación estándar y frecuencia 
lista=datos['Rango'].values.tolist()
promedio=[]
desv=[]
dif=[]
N=[]
for i in lista:
  lista2=list(sorted(i))
  a=[]
  for j in lista2:
    if lista2.index(j) >= 1:
        a.append(j-lista2[lista2.index(j)-1])
  promedio.append(np.mean(a))
  desv.append(np.std(a))
  dif.append(a)
  N.append(len(a))
  

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in true_divide
  subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
#vectores con los resultados
resultados=[dif,promedio,desv,N]

In [ ]:
#colocar el nombre a las columnas
calculos=pd.DataFrame(resultados).T
calculos.columns=['Diferencia de Fechas','Frecuencia de compra','Desviación estándar','Cantidad de elementos']

In [ ]:
#union de los calculos en una base final
baseFinal=pd.merge(datos[['Solic','FechaInicio','FechaFinal']],calculos,left_index=True,right_index=True,how='outer')
baseFinal.head()

,Solic,FechaInicio,FechaFinal,Diferencia de Fechas,Frecuencia de compra,Desviación estándar,Cantidad de elementos
0,49128.0,2021-11-02,2022-07-11,"[7.0, 1.0, 6.0, 7.0, 2.0, 4.0, 7.0, 33.0, 10.0...",10.913043,15.901635,23
1,69792.0,2021-11-05,2022-07-05,"[11.0, 3.0, 11.0, 3.0, 14.0, 4.0, 17.0, 4.0, 1...",12.1,9.585927,20
2,69838.0,2022-07-12,2022-07-12,[],NaN,NaN,0
3,69868.0,2021-11-09,2022-07-05,"[14.0, 13.0, 8.0, 7.0, 22.0, 13.0, 14.0, 28.0,...",17.0,8.392173,14
4,70334.0,2022-02-26,2022-05-31,"[32.0, 62.0]",47.0,15.0,2


In [74]:
baseFinal['fuga'] = baseFinal.apply(lambda x: 1 if any(x['Diferencia de Fechas'])>=60   else 0, axis = 1)

In [79]:
baseFinal.head(10)

,Solic,FechaInicio,FechaFinal,Diferencia de Fechas,Frecuencia de compra,Desviación estándar,Cantidad de elementos,fuga
0,49128.0,2021-11-02,2022-07-11,"[7.0, 1.0, 6.0, 7.0, 2.0, 4.0, 7.0, 33.0, 10.0...",10.913043,15.901635,23,0
1,69792.0,2021-11-05,2022-07-05,"[11.0, 3.0, 11.0, 3.0, 14.0, 4.0, 17.0, 4.0, 1...",12.1,9.585927,20,0
2,69838.0,2022-07-12,2022-07-12,[],NaN,NaN,0,0
3,69868.0,2021-11-09,2022-07-05,"[14.0, 13.0, 8.0, 7.0, 22.0, 13.0, 14.0, 28.0,...",17.0,8.392173,14,0
4,70334.0,2022-02-26,2022-05-31,"[32.0, 62.0]",47.0,15.0,2,0
5,70430.0,2021-11-06,2022-07-02,"[4.0, 7.0, 5.0, 5.0, 4.0, 6.0, 15.0, 17.0, 4.0...",11.333333,12.855732,21,0
6,70566.0,2021-11-03,2022-07-06,"[7.0, 7.0, 7.0, 21.0, 3.0, 32.0, 3.0, 32.0, 63...",18.846154,17.627977,13,0
7,71438.0,2021-11-04,2022-07-14,"[14.0, 14.0, 7.0, 7.0, 7.0, 14.0, 14.0, 7.0, 7...",18.0,17.004201,14,0
8,72510.0,2022-06-10,2022-06-10,[],NaN,NaN,0,0
9,73010.0,2021-11-03,2022-06-29,"[28.0, 6.0, 8.0, 28.0, 14.0, 21.0, 70.0, 28.0,...",23.8,17.2093,10,0


In [78]:
np.count_nonzero(baseFinal['fuga'])

0        0
1        0
2        0
3        0
4        0
        ..
79202    0
79203    0
79204    0
79205    0
79206    0
Name: fuga, Length: 79207, dtype: int64

**Clasificador bosques aleatorios**

In [81]:
#partición de los datos en entrenamiento y prueba
from sklearn.model_selection import train_test_split
X=baseFinal['Solic']
y=baseFinal['fuga']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [82]:
#corriendo el clasificador
from sklearn.ensemble import RandomForestClassifier
algoritmo = RandomForestClassifier(n_estimators = 10, criterion = 'entropy')

In [83]:
#Entreno el modelo
algoritmo.fit(X_train, y_train)

ValueError: ignored

In [ ]:
#Realizo una predicción
y_pred = algoritmo.predict(X_test)

In [ ]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred)
print('Precisión del modelo:')
print(precision)

**Clasificador KNN (K vecinos más cercano)**

In [ ]:
#división en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#eleccion del mejor k
k_range = range(1, 20)
scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train, y_train)
    scores.append(knn.score(X_test, y_test))
plt.figure()
plt.xlabel('k')
plt.ylabel('accuracy')
plt.scatter(k_range, scores)
plt.xticks([0,5,10,15,20])


In [ ]:

n_neighbors = 7

knn = KNeighborsClassifier(n_neighbors)
knn.fit(X_train, y_train)
print('Accuracy of K-NN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))
print('Accuracy of K-NN classifier on test set: {:.2f}'
     .format(knn.score(X_test, y_test)))

**Clasificador maquinas de soporte vectorial**

In [ ]:
# división de los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=109) 

In [ ]:
#maquinas de soporte 
#creación del clasificador
clf = svm.SVC(kernel='linear') # Linear Kernel

#Entrenamiento del modelo
clf.fit(X_train, y_train)

#Predecir la respuesta para el conjunto de datos de prueba
y_pred = clf.predict(X_test)

In [ ]:
# Precisión del modelo
print("Exactitud:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
# Precisión del modelo: ¿qué porcentaje de tuplas positivas se etiquetan como tales?
print("Precisión:",metrics.precision_score(y_test, y_pred))

# Exhaustividad
print("Exhaustividad:",metrics.recall_score(y_test, y_pred))

Lo que tenia pensado era revisar que modelo se ajusto mejor a los datos de acuerdo a las metricas y con ese modelo predecir que clientes tienen mas probabilidad de fuga y luego exportarlo a excel para hacer un cruce y determinar que clientes potenciales se han fugado.